In [1]:
from TelopsToolbox.hcc.readIRCam import read_ircam
from TelopsToolbox.hcc.writeIRCam import write_ircam
import TelopsToolbox.utils.image_processing as ip
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import biosig 

In [6]:
way = '/Users/lubimyj/Git/experiments/Plansee-Tungstan-W_cover/'
materials = ['Denal920','DX2']
specimens = ['1','2','3','4','5','6']
subFold = '/IRCamera/Specimen'
to_IR_data = '/Specimen'
IR_index = '_IR/'
testFold = 'test/EXP #1/'
dataFold = '/FLT/'
files = ['Parameters.txt','incid.FLT','trans.FLT']

material = materials[1]
specimen = specimens[5]

tr_c = 0.9

path = way+material+subFold+specimen+to_IR_data+specimen+IR_index
#path == '/Users/lubimyj/Git/experiments/Plansee-Tungstan-W/Denal920/IRCamera/Specimen1/Specimen1_IR/'
file = to_IR_data[1:]+specimen+IR_index[:-1]+'.hcc'
file[:-3]

'Specimen6_IR.'

In [7]:
""" read the first 50 frames of a sequence hcc """
data, header, specialPixel, nonSpecialPixel = read_ircam(path+file)
                                                        #frames=list(range(0, 50)))

header_df = pd.DataFrame(header)
frame = data[0]
len(data)

70

In [8]:
""" look for a particular header """
numero_header = 0
this_header = header_df.iloc[numero_header]

sr = this_header[20]



In [9]:
def bin_image(image, tresh = 5):
    image = [list(map(lambda x: 1 if (x > tresh) else 0, line)) for line in image]
    return image
    





In [10]:
""" print a frame """
image_2D = ip.form_image(this_header, data[0])
first_frame = image_2D[0]-273.15

n = 69


image_2D = ip.form_image(this_header, data[n])
cur = image_2D[0]-273.15
c_lim = ip.image_scaling_limits(frame)
vmin, vmax = c_lim.squeeze()
res = (cur-first_frame)*bin_image(cur, cur.max()*tr_c)
fig1 = make_subplots(rows=2, cols=1)

max_x, max_y = np.unravel_index(np.argmax(res, axis=None), res.shape)
fig1.update_layout(template = 'plotly_white', width=1000, height=800)
fig1.add_trace(go.Heatmap(z = cur), row = 2, col =1)
fig1.add_trace(go.Heatmap(z = res), row = 1, col =1)

fig1.update_layout(title = f'{material}, specimen {specimen} time {round(n/sr,6)}')
fig1.add_annotation(x=max_y,
                    y=max_x,
                    text = f'Max_value {round(res[max_x][max_y],2)}',
                    bgcolor="#ffffff")

#fig1.update_layout(template = 'plotly_white', width=800, height=600)
fig1.show()

In [11]:
max_y

35

In [16]:
movie = [ip.form_image(this_header, item)[0]-273.15 for item in data]
zeroed_movie = [item - first_frame for item in movie]
masks = [bin_image(item, item.max()*tr_c) for item in movie]
bin_movie = [zeroed_movie[item]*masks[item] for item in range(len(masks))]

def make_movie(bin_movie):
    fig_dict = {
        "data": [],
        "layout": { 'yaxis': dict(domain=[0.25, 1]),
                    'yaxis2': dict(domain=[0, 0.2]),
                    'width' : 1000,
                    'height': 800,
                    'title': 'Matetial: ' + material + ' specimen №' + specimen
                    },
        "frames": [],
        "annotation": {}
    }


    fig_dict["layout"]["hovermode"] = "closest"
    fig_dict["layout"]["updatemenus"] = [
        {
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 500, "redraw": True},
                                    "fromcurrent": True, "transition": {"duration": 300,
                                                                        "easing": "quadratic-in-out"}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": True},
                                    "mode": "immediate",
                                    "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }
    ]

    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    }

    #make data
    first_image = bin_movie[0]

    frame_zero = go.Heatmap(z = first_image)
    plot_zero = go.Scatter(y = np.array(first_image).max(axis=0), yaxis ='y2')

    fig_dict['data'].append(frame_zero)
    fig_dict['data'].append(plot_zero)


    # make frames
    for pic in range(len(bin_movie)):
        
        current_image = bin_movie[pic]
        vertical_temp = np.array(current_image).max(axis=0)

        trace2 = go.Scatter(y = vertical_temp, yaxis ='y2',)
        trace1 = go.Heatmap(z = current_image, yaxis="y1")

        frame_n = {"data": [trace1, trace2], "name": str(pic)}

        slider_step = {"args": [[pic],
                                {"data": {"duration": 300, "redraw": False},
                                "mode": "immediate",
                                "transition": {"duration": 300}}
                                ],
                        "label": f'{round(pic/this_header[20]*1e+6,3)} mks',
                        "method": "animate"}
                    
        fig_dict['layout']['yaxis2']['range'] = [0,max(vertical_temp)]
        fig_dict["frames"].append(frame_n)   
        sliders_dict["steps"].append(slider_step)

    fig_dict["layout"]["sliders"] = [sliders_dict]

    fig = go.Figure(fig_dict)
    return fig


fig = make_movie(bin_movie)
fig.show()

In [15]:
temperature_data = pd.DataFrame({'time': np.array(range(len(data)))/this_header[20]})
temperature_area = []
temperature_line = []
temperature_point = []
time = []
for tres in range(10):
    sample = []
    for pic in range(len(data)):

        image_2D = ip.form_image(this_header, data[pic])[0]-273.15
        mask = bin_image(image_2D, image_2D.max()*(tres*0.1))
        final_image = (image_2D-first_frame)*mask
        if np.array(mask).sum():
            sample.append(final_image.sum()/np.array(mask).sum())
        else:
            sample.append(0)
        time.append(pic/this_header[20])


    temperature_area.append(sample)
    temperature_data[f'Tr_{round(tres*0.1,2)}'] = sample

for pic in range(len(data)):
    image_2D = ip.form_image(this_header, data[pic])[0]-273.15 - first_frame
    temperature_line.append(image_2D[10].sum()/64)
    temperature_point.append(image_2D.max())
temperature_data['Line'] = temperature_line
temperature_data['Point'] = temperature_point



    

In [16]:
#temperature_data.to_csv(path+'Temperature_Spec_'+specimen+'.csv', sep = '\t')
mask = bin_image(image_2D, image_2D.min()+image_2D.max()*tres*0.1)

In [20]:
fig2 = go.Figure()
fig2.update_layout(template = 'plotly_white', width=800, height=600,
                  title = 'Analysis of trshold level infl')

fig2.update_yaxes(title = 'Temperature rise. C')
fig2.update_xaxes(title = 'Time, s')
for sample in range(10):
    trace = go.Scatter(x = time, y = temperature_area[sample], name = f'Trs = {round(sample*0.1,2)}')
    fig2.add_trace(trace)
fig2.add_trace(go.Scatter(x = time, y = temperature_point, name = 'Max_value'))
fig2.show()

In [21]:
fig3 = go.Figure()
fig3.update_layout(template = 'plotly_white', width=800, height=600,
                  title = f'Delta Temperature during Experiment {material} Specimen {specimen}')

fig3.update_yaxes(title = 'Temperature')
fig3.update_xaxes(title = 'Time')
trace_area = go.Scatter(x = time, y = temperature_area[6], name = f'Mean Temperature')
trace_point = go.Scatter(x = time, y = temperature_point, name = 'Max Temperarure')

fig3.add_trace(trace_area)
fig3.add_trace(trace_point)
fig3.show()